#  MODEL PRESELECTION - ADDENDUM: TRAINING PREDICTIONS

In this addendum Notebook we load the optimized classifiers and calculate and store the predictions for the training set.

## Modules and configuration

### Modules

In [2]:
import pandas as pd
import numpy as np

#import copy

#from time import time

#from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, \
#    f1_score, log_loss, matthews_corrcoef, classification_report, \
#    get_scorer_names, confusion_matrix

from collections import OrderedDict

import warnings

#from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif

#from sklearn.gaussian_process.kernels import RBF, RationalQuadratic, DotProduct

#from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit # Se usa para evitar overfitting
#from sklearn.model_selection import cross_validate #### NOTA, IGUAL ES MEJOR ESTE, PARA TENER EL CONTROL Y SACAR
# TODOS LOS RUNS QUE QUERAMOS
###### NO HACE FALTA EL GRID SEARCH NI NADA DE ESTO??? SIMPLEMENTE LE HACEMOS EL "FIT" Y LUEGO MEDIMOS CON
# EL "PREDICT" SOBRE EL VALIDATION SET

from sklearn.linear_model import Perceptron, LogisticRegression, PassiveAggressiveClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier, HistGradientBoostingClassifier

import pickle

#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})

from IPython.display import display

# from imblearn import 

### Configuration

In [3]:
RANDOM_STATE = 11 # For reproducibility

S4_TRAIN_SET_IN = "../data/DATASETS_ML/S4_02_DS_AfterImputing_TrainTest_OVERSAMPLED_n3.csv"
# Train/test set for S4 sample, all 112 features
#S4_VALIDATION_SET_IN = "../data/DATASETS_ML/S4_02_DS_AfterImputing_Validation.csv"
# Validation set for S4 sample, all 112 features

REL_FEATURES_IN = "../data/ML_MODELS/ML_pipeline_steps/Reliable_features.pickle"
#UNREL_FEATURES_IN = "../data/ML_MODELS/ML_pipeline_steps/Unreliable_features.pickle"

FITTED_CLF_IN = "OVERSAMPLED_fitted_clf_opt_n3.pickle"

ML_ADD_COLUMNS = ['Karmn'] # Only cesium features and this column will be kept.
S4_ADD_COLUMNS = ['ID', 'Pulsating', 'frequency', 'amplitudeRV',
                  'offsetRV', 'refepochRV', 'phase'] # Only cesium features and these columns will be kept.

MODELS_FOLDER = "../data/ML_MODELS/ML_model_preselection/"

#PRECISION_RESULTS_OUT = "ModelPreselection_PrecisionResults_OversampledSMOTE.csv"
TRAIN_PREDICTIONS_OUT = "ModelPreselection_TrainPredictions_OversampledSMOTE_n3.csv"

#IMAGE_FOLDER = './img/'

### Functions

## Load data

We load the data, which are the time series as previously featurized by _cesium_, scaled, and with `NaN` values imputed by a `KNNImputer`.

### Load reliable features list

In [4]:
rel_features = pickle.load(open(REL_FEATURES_IN, 'rb'))
print(rel_features)

['all_times_nhist_numpeaks', 'all_times_nhist_peak1_bin', 'all_times_nhist_peak2_bin', 'all_times_nhist_peak3_bin', 'all_times_nhist_peak4_bin', 'all_times_nhist_peak_1_to_2', 'all_times_nhist_peak_1_to_3', 'all_times_nhist_peak_1_to_4', 'all_times_nhist_peak_2_to_3', 'all_times_nhist_peak_2_to_4', 'all_times_nhist_peak_3_to_4', 'all_times_nhist_peak_val', 'avg_double_to_single_step', 'cad_probs_1', 'cad_probs_10', 'cad_probs_20', 'cad_probs_30', 'cad_probs_40', 'cad_probs_50', 'cad_probs_100', 'cad_probs_500', 'cad_probs_1000', 'cad_probs_5000', 'cad_probs_10000', 'cad_probs_50000', 'cad_probs_100000', 'cad_probs_500000', 'cad_probs_1000000', 'cad_probs_5000000', 'cad_probs_10000000', 'cads_avg', 'cads_med', 'cads_std', 'med_double_to_single_step', 'n_epochs', 'std_double_to_single_step', 'total_time', 'percent_beyond_1_std', 'freq1_rel_phase2', 'freq1_rel_phase3', 'freq1_rel_phase4', 'freq2_rel_phase2', 'freq2_rel_phase3', 'freq2_rel_phase4', 'freq3_rel_phase2', 'freq3_rel_phase3', '

###  Load the oversampled S4 sample data train set

In [5]:
s4_tr = pd.read_csv(S4_TRAIN_SET_IN, sep=',', decimal='.')
s4_tr

,Pulsating,all_times_nhist_numpeaks,all_times_nhist_peak1_bin,all_times_nhist_peak2_bin,all_times_nhist_peak3_bin,all_times_nhist_peak4_bin,all_times_nhist_peak_1_to_2,all_times_nhist_peak_1_to_3,all_times_nhist_peak_1_to_4,all_times_nhist_peak_2_to_3,...,freq1_rel_phase2,freq1_rel_phase3,freq1_rel_phase4,freq2_rel_phase2,freq2_rel_phase3,freq2_rel_phase4,freq3_rel_phase2,freq3_rel_phase3,freq3_rel_phase4,freq_model_phi1_phi2
0,0,-0.674126,0.519174,0.466681,0.766297,1.786498,-0.304944,0.843252,0.189055,1.390901,...,0.646033,0.908818,1.305379,1.413989,0.174334,-0.188773,0.985693,-0.258841,-1.099919,-0.461571
1,1,-1.626729,1.911247,-0.740748,0.691384,0.168331,1.522002,1.166420,0.157675,0.019744,...,1.532902,-1.224350,0.710232,-1.272791,1.617586,1.392776,0.260283,0.708876,1.030413,0.400968
2,0,-0.039057,-1.012107,0.013895,-0.357397,1.168762,-0.232282,-0.443941,-0.136007,-0.412519,...,0.384058,0.882515,1.044322,-1.204443,-0.593335,-1.011092,0.592500,0.135213,0.725303,-0.319881
3,0,-0.039057,1.632833,-0.514355,0.166993,1.477630,-0.544204,-0.572606,-0.586661,-0.338658,...,1.457441,-0.921750,-1.095322,-0.031960,-0.068737,1.152465,-0.672518,0.391616,-1.301501,0.559262
4,0,0.596012,-0.176863,-1.042605,-0.432310,0.242158,-0.277263,-0.498198,-0.370020,-0.451106,...,-0.829296,-0.057582,0.541269,1.529879,-0.689578,1.475290,-1.268004,1.297212,1.327633,-0.059789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,1,0.473173,-0.668664,0.135630,-0.259518,-1.373200,-0.470559,-0.589896,-0.548709,-0.448373,...,-0.038051,-0.958444,0.386309,-0.805888,0.697947,0.293784,-0.667106,-0.192877,1.224979,-1.394465
1340,1,-0.993662,-0.524004,0.918516,1.589867,-0.372796,-0.377095,-0.318659,-0.118490,-0.120285,...,0.036617,1.595782,0.586577,-0.903790,0.315474,-0.690108,-0.846619,0.819517,-0.407266,-1.583849
1341,1,0.610021,-1.068148,0.492761,0.150468,-0.980256,-0.169826,-0.341273,-0.177918,-0.341825,...,-0.194276,-0.105307,1.257649,-0.697622,-0.195618,-0.123771,-0.670504,0.582847,1.003724,-1.285203
1342,1,1.061356,-0.373715,0.099776,-0.660900,-1.102506,0.339215,-0.083127,0.190607,-0.361995,...,-0.124422,-0.875222,1.033421,-0.335086,-0.822268,-0.909756,-0.608352,-0.050957,1.095146,-1.091260


Notice that target variable is already encoded as `0` / `1` and that we now only have the features (i.e. the metadata are no longer present).

## Load the trained classifiers

In [6]:
fitted_classifiers_opt = pickle.load(open(MODELS_FOLDER + FITTED_CLF_IN, 'rb'))
fitted_classifiers_opt

OrderedDict([('Perceptron',
              OrderedDict([('Fitted_clf',
                            Perceptron(alpha=0.001, l1_ratio=0.075, max_iter=500, penalty='l1',
                                       random_state=11)),
                           ('OptTrain_time', 3.007888078689575),
                           ('Training metrics',
                            OrderedDict([('accuracy', 0.6019345238095238),
                                         ('precision', 0.6203866432337434),
                                         ('recall', 0.5252976190476191),
                                         ('F1', 0.5688960515713134),
                                         ('log_loss', 14.34773404998339),
                                         ('MCC', 0.2063068189451933)])),
                           ('Validation metrics',
                            OrderedDict([('accuracy', 0.616),
                                         ('precision', 0.09302325581395349),
                                  

## Predictions on the training dataset

### Predictions

We now save the predictions on the validation dataset, alongside with all available metadata, so that they can be analysed later on.

In [8]:
s4_tr_w_pred = s4_tr.copy()
s4_tr_w_pred

,Pulsating,all_times_nhist_numpeaks,all_times_nhist_peak1_bin,all_times_nhist_peak2_bin,all_times_nhist_peak3_bin,all_times_nhist_peak4_bin,all_times_nhist_peak_1_to_2,all_times_nhist_peak_1_to_3,all_times_nhist_peak_1_to_4,all_times_nhist_peak_2_to_3,...,freq1_rel_phase2,freq1_rel_phase3,freq1_rel_phase4,freq2_rel_phase2,freq2_rel_phase3,freq2_rel_phase4,freq3_rel_phase2,freq3_rel_phase3,freq3_rel_phase4,freq_model_phi1_phi2
0,0,-0.674126,0.519174,0.466681,0.766297,1.786498,-0.304944,0.843252,0.189055,1.390901,...,0.646033,0.908818,1.305379,1.413989,0.174334,-0.188773,0.985693,-0.258841,-1.099919,-0.461571
1,1,-1.626729,1.911247,-0.740748,0.691384,0.168331,1.522002,1.166420,0.157675,0.019744,...,1.532902,-1.224350,0.710232,-1.272791,1.617586,1.392776,0.260283,0.708876,1.030413,0.400968
2,0,-0.039057,-1.012107,0.013895,-0.357397,1.168762,-0.232282,-0.443941,-0.136007,-0.412519,...,0.384058,0.882515,1.044322,-1.204443,-0.593335,-1.011092,0.592500,0.135213,0.725303,-0.319881
3,0,-0.039057,1.632833,-0.514355,0.166993,1.477630,-0.544204,-0.572606,-0.586661,-0.338658,...,1.457441,-0.921750,-1.095322,-0.031960,-0.068737,1.152465,-0.672518,0.391616,-1.301501,0.559262
4,0,0.596012,-0.176863,-1.042605,-0.432310,0.242158,-0.277263,-0.498198,-0.370020,-0.451106,...,-0.829296,-0.057582,0.541269,1.529879,-0.689578,1.475290,-1.268004,1.297212,1.327633,-0.059789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1339,1,0.473173,-0.668664,0.135630,-0.259518,-1.373200,-0.470559,-0.589896,-0.548709,-0.448373,...,-0.038051,-0.958444,0.386309,-0.805888,0.697947,0.293784,-0.667106,-0.192877,1.224979,-1.394465
1340,1,-0.993662,-0.524004,0.918516,1.589867,-0.372796,-0.377095,-0.318659,-0.118490,-0.120285,...,0.036617,1.595782,0.586577,-0.903790,0.315474,-0.690108,-0.846619,0.819517,-0.407266,-1.583849
1341,1,0.610021,-1.068148,0.492761,0.150468,-0.980256,-0.169826,-0.341273,-0.177918,-0.341825,...,-0.194276,-0.105307,1.257649,-0.697622,-0.195618,-0.123771,-0.670504,0.582847,1.003724,-1.285203
1342,1,1.061356,-0.373715,0.099776,-0.660900,-1.102506,0.339215,-0.083127,0.190607,-0.361995,...,-0.124422,-0.875222,1.033421,-0.335086,-0.822268,-0.909756,-0.608352,-0.050957,1.095146,-1.091260


In [9]:
warnings.simplefilter('ignore')
for classifier in list(fitted_classifiers_opt.keys()):
    print("Calculating predictions for classifier %s..." %classifier)
    # Training set:
    y_tr_pred = fitted_classifiers_opt[classifier]['Fitted_clf'].predict(s4_tr[rel_features])
    s4_tr_w_pred['Prediction_' + classifier] = y_tr_pred

Calculating predictions for classifier Perceptron...
Calculating predictions for classifier LogisticRegression...
Calculating predictions for classifier PassiveAggressiveClassifier...
Calculating predictions for classifier SVC...
Calculating predictions for classifier KNeighborsClassifier...
Calculating predictions for classifier GaussianProcessClassifier...
Calculating predictions for classifier DecisionTreeClassifier...
Calculating predictions for classifier RandomForestClassifier...
Calculating predictions for classifier AdaBoostClassifier...
Calculating predictions for classifier GradientBoostingClassifier...


In [10]:
s4_tr_w_pred.head(20)

,Pulsating,all_times_nhist_numpeaks,all_times_nhist_peak1_bin,all_times_nhist_peak2_bin,all_times_nhist_peak3_bin,all_times_nhist_peak4_bin,all_times_nhist_peak_1_to_2,all_times_nhist_peak_1_to_3,all_times_nhist_peak_1_to_4,all_times_nhist_peak_2_to_3,...,Prediction_Perceptron,Prediction_LogisticRegression,Prediction_PassiveAggressiveClassifier,Prediction_SVC,Prediction_KNeighborsClassifier,Prediction_GaussianProcessClassifier,Prediction_DecisionTreeClassifier,Prediction_RandomForestClassifier,Prediction_AdaBoostClassifier,Prediction_GradientBoostingClassifier
0,0,-0.674126,0.519174,0.466681,0.766297,1.786498,-0.304944,0.843252,0.189055,1.390901,...,0,1,1,0,0,0,0,0,1,1
1,1,-1.626729,1.911247,-0.740748,0.691384,0.168331,1.522002,1.166420,0.157675,0.019744,...,1,1,1,1,1,1,1,1,0,0
2,0,-0.039057,-1.012107,0.013895,-0.357397,1.168762,-0.232282,-0.443941,-0.136007,-0.412519,...,1,1,1,1,0,1,0,0,0,0
3,0,-0.039057,1.632833,-0.514355,0.166993,1.477630,-0.544204,-0.572606,-0.586661,-0.338658,...,0,0,0,0,0,0,0,0,0,0
4,0,0.596012,-0.176863,-1.042605,-0.432310,0.242158,-0.277263,-0.498198,-0.370020,-0.451106,...,1,1,1,0,0,0,0,0,0,0
5,0,-1.626729,1.911247,-0.740748,0.616471,-0.110402,-0.563300,-0.530210,-0.405716,-0.254488,...,1,1,1,0,0,0,0,0,0,1
6,0,1.231081,-0.594485,-1.118070,0.616471,-0.143927,-0.581754,-0.275405,-0.442036,0.167407,...,1,1,1,0,0,0,0,0,0,0
7,0,1.231081,-0.594485,-1.042605,-0.282484,-0.530012,-0.503948,-0.424345,-0.581960,-0.150469,...,0,0,0,0,0,0,0,0,0,0
8,1,-0.674126,-0.733692,0.919467,1.215775,-0.066710,-0.456588,0.673164,0.350662,1.419767,...,0,0,1,1,1,1,0,1,0,0
9,1,-0.674126,-0.524881,0.013895,0.691384,0.396592,-0.466417,1.196717,2.700897,2.213595,...,1,1,1,1,1,1,1,1,1,0


### Prediction probabilities (if available)

In [11]:
warnings.simplefilter('ignore')
for classifier in list(fitted_classifiers_opt.keys()):
    print("Calculating predictions for classifier %s..." %classifier)
    # Validation set:
    try:
        y_tr_pred_proba = fitted_classifiers_opt[classifier]['Fitted_clf'].predict_proba(s4_tr[rel_features])
        s4_tr_w_pred['PredictionProb_' + classifier] = pd.Series(y_tr_pred_proba[:, 1])
        print("... ok, probabilities calculated")
    except:
        print("**WARNING: 'predict_proba' method failed for classifier '%s'." %classifier)
        s4_tr_w_pred['PredictionProb_' + classifier] = np.nan

Calculating predictions for classifier Perceptron...
**WARNING: 'predict_proba' method failed for classifier 'Perceptron'.
Calculating predictions for classifier LogisticRegression...
... ok, probabilities calculated
Calculating predictions for classifier PassiveAggressiveClassifier...
**WARNING: 'predict_proba' method failed for classifier 'PassiveAggressiveClassifier'.
Calculating predictions for classifier SVC...
**WARNING: 'predict_proba' method failed for classifier 'SVC'.
Calculating predictions for classifier KNeighborsClassifier...
... ok, probabilities calculated
Calculating predictions for classifier GaussianProcessClassifier...
... ok, probabilities calculated
Calculating predictions for classifier DecisionTreeClassifier...
... ok, probabilities calculated
Calculating predictions for classifier RandomForestClassifier...
... ok, probabilities calculated
Calculating predictions for classifier AdaBoostClassifier...
... ok, probabilities calculated
Calculating predictions for cl

In [12]:
s4_tr_w_pred.head(20)

,Pulsating,all_times_nhist_numpeaks,all_times_nhist_peak1_bin,all_times_nhist_peak2_bin,all_times_nhist_peak3_bin,all_times_nhist_peak4_bin,all_times_nhist_peak_1_to_2,all_times_nhist_peak_1_to_3,all_times_nhist_peak_1_to_4,all_times_nhist_peak_2_to_3,...,PredictionProb_Perceptron,PredictionProb_LogisticRegression,PredictionProb_PassiveAggressiveClassifier,PredictionProb_SVC,PredictionProb_KNeighborsClassifier,PredictionProb_GaussianProcessClassifier,PredictionProb_DecisionTreeClassifier,PredictionProb_RandomForestClassifier,PredictionProb_AdaBoostClassifier,PredictionProb_GradientBoostingClassifier
0,0,-0.674126,0.519174,0.466681,0.766297,1.786498,-0.304944,0.843252,0.189055,1.390901,...,NaN,0.667095,NaN,NaN,0.0,0.363527,0.071429,0.295779,0.501447,0.549685
1,1,-1.626729,1.911247,-0.740748,0.691384,0.168331,1.522002,1.166420,0.157675,0.019744,...,NaN,0.715795,NaN,NaN,1.0,0.676525,1.000000,0.614086,0.496762,0.458713
2,0,-0.039057,-1.012107,0.013895,-0.357397,1.168762,-0.232282,-0.443941,-0.136007,-0.412519,...,NaN,0.634074,NaN,NaN,0.0,0.537343,0.000000,0.183967,0.494150,0.302085
3,0,-0.039057,1.632833,-0.514355,0.166993,1.477630,-0.544204,-0.572606,-0.586661,-0.338658,...,NaN,0.347344,NaN,NaN,0.0,0.434809,0.066667,0.234851,0.499231,0.393730
4,0,0.596012,-0.176863,-1.042605,-0.432310,0.242158,-0.277263,-0.498198,-0.370020,-0.451106,...,NaN,0.568765,NaN,NaN,0.0,0.341284,0.000000,0.140025,0.489992,0.326507
5,0,-1.626729,1.911247,-0.740748,0.616471,-0.110402,-0.563300,-0.530210,-0.405716,-0.254488,...,NaN,0.657895,NaN,NaN,0.0,0.361038,0.090909,0.279135,0.498291,0.549455
6,0,1.231081,-0.594485,-1.118070,0.616471,-0.143927,-0.581754,-0.275405,-0.442036,0.167407,...,NaN,0.668796,NaN,NaN,0.0,0.372359,0.000000,0.193979,0.488560,0.331416
7,0,1.231081,-0.594485,-1.042605,-0.282484,-0.530012,-0.503948,-0.424345,-0.581960,-0.150469,...,NaN,0.373864,NaN,NaN,0.0,0.246883,0.000000,0.165939,0.397113,0.339790
8,1,-0.674126,-0.733692,0.919467,1.215775,-0.066710,-0.456588,0.673164,0.350662,1.419767,...,NaN,0.404799,NaN,NaN,1.0,0.607824,0.375000,0.527092,0.495564,0.440255
9,1,-0.674126,-0.524881,0.013895,0.691384,0.396592,-0.466417,1.196717,2.700897,2.213595,...,NaN,0.617399,NaN,NaN,1.0,0.767806,1.000000,0.720505,0.502588,0.451316


### Save the predictions

And we now save the file:

In [13]:
s4_tr_w_pred.to_csv(MODELS_FOLDER + TRAIN_PREDICTIONS_OUT, sep=',', decimal='.', index=False)

## Summary

**RESULTS:**

- We tested different classifiers from different families against the S4 sample oversampled with SMOTE.
- In general, results show slight improvement with a very simple and naive model hyperparameter optimization.
- Overfitting seems to be a serious problem, more specially with the tree / ensemble methods.
- We have stored both the precision of the different ML models, as well as their predictions on the validation set (and prediction probabilities when available).

**CONCLUSIONS:**

- Additional work in tree and ensemble classifiers is needed to prevento overfitting: pruning the trees, for example (even if some values for `ccp_alpha` parameter were tried as part of the optimization).
